In [1]:
import pykov
import json

In [2]:
word_matrices = {}

In [3]:
data = json.load(open('geekhub_off-topic.json')) # load data

In [4]:
# reformat data
messages = []
for mes in data['messages']:
    messages.append((mes['author']['name'], mes['content']))

In [5]:
Musers = pykov.Matrix()
for mes, nxt in zip(messages[:-1], messages[1:]):
    if (mes[0], nxt[0]) not in Musers: Musers[(mes[0], nxt[0])] = 0
    Musers[(mes[0], nxt[0])] = Musers[(mes[0], nxt[0])] + 1
Musers.stochastic()

In [6]:
def printmessages(n):
    user = pykov.Vector({'Lenny':1}) # starting point for markov chain
    for _ in range(n):
        choice = (user * Musers).choose() # choose next user to write a message
        user = pykov.Vector({choice: 1}) # create pykov vector for future user calculation

        if choice in word_matrices:
            Mword = word_matrices[choice]
        else: # create word matrix
            Mword = pykov.Matrix({(1, 1) : 1}) # create matrix for word chain
            for mes in [m for m in messages if m[0] == choice]: # add every word to matrix
                splt = mes[1].split(' ')
                if (0, splt[0]) not in Mword: Mword[(0, splt[0])] = 0
                Mword[(0, splt[0])] += 1
                for word, nxt in zip(splt[:-1], splt[1:]):
                    if (word, nxt) not in Mword: Mword[(word, nxt)] = 0
                    Mword[(word, nxt)] += 1
                if (splt[-1], 1) not in Mword: Mword[(splt[-1], 1)] = 0
                Mword[(splt[-1], 1)] += 1
            Mword.stochastic()
            word_matrices[choice] = Mword
            
        C = pykov.Chain(Mword)
        print(choice + ":", " ".join(C.walk(100, 0, 1)[1:-1]))

In [7]:
printmessages(0)